In [ ]:
# based on https://towardsdatascience.com/artificial-neural-networks-optimization-using-genetic-algorithm-with-python-1fe8ed17733e
# different crossover function: mean
from common import Trial, safe_log, nll, load_df, df_to_ML_data, timed_method, ts


from IPython.core.display import Javascript
from IPython.display import display
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy as np
import pickle
import keras
from keras import backend as K
from keras.models import model_from_json
from keras.layers import Dense, Flatten, BatchNormalization, Dropout, Lambda
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Sequential
from keras.metrics import binary_accuracy
from sklearn.metrics import accuracy_score
import keras_metrics
from keras import metrics
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
import matplotlib
import matplotlib.pyplot
import logging
import random
import sys


logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',level=logging.INFO, 
                    filename='genetic_learning_weights_only_kernel_' + ts() + '.log', filemode='w+')


log = logging.getLogger(__name__)

config = tf.ConfigProto()#device_count = {'GPU': 0})
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

sess = tf.Session()


In [ ]:
with open('json_weights_shallow.pkl', 'rb') as f:
    model_topology_json, weights = pickle.load(f)

(model_topology_json, weights)

In [ ]:
X_train, X_test, y_train, y_test = df_to_ML_data(load_df())

In [ ]:
def check_weights(a, b):
    assert len(a) == len(b)
    for i in range(len(a)):
        assert a[i].shape ==  b[i].shape

def create_model_programatically(weights = None):
    input_shape = (64, 256, 1)
    num_classes = 2
    
    model = Sequential()
    model.add(Conv2D(30, kernel_size=(1, 25),
                     input_shape=input_shape))
    model.add(Conv2D(10, kernel_size=(64, 1)))
    model.add(Lambda(lambda x: x ** 2))
    model.add(AveragePooling2D(pool_size=(1, 15), strides=(1, 1)))
    model.add(Lambda(lambda x: safe_log(x)))
    model.add(Conv2D(2, kernel_size=(1, 8), dilation_rate=(15, 1)))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    if weights != None:
        #check_weights(model.get_weights(), weights)

        model.set_weights(weights)

    
    return model

def individual_fitness(model_topology_json, model_weights, X, y):
    keras_model = create_model_programatically(model_weights)
    y_pred = keras_model.predict(X, batch_size=512)
    
    m = tf.keras.metrics.BinaryAccuracy()
    m.update_state(y, y_pred)

    return K.mean(binary_accuracy(y, y_pred)).eval(session=K.get_session())
    
def population_fitness(model_topology_json, population_of_models, X, y):
    fitness_scores = []
    
    for model_weights in population_of_models:#tqdm(population_of_models, desc='Current Population Fitness calculation progress', position=2):
        fitness_scores.append(individual_fitness(model_topology_json, model_weights, X, y))
                          
    return fitness_scores

def fittest_parents_of_generation(population, fitness_scores, num_parents, selector = np.argmin):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = []
    for _ in range(num_parents - 1):
        best_fitness_idx = selector(fitness_scores)
        parents.append(population[best_fitness_idx])
        del population[best_fitness_idx]
        del fitness_scores[best_fitness_idx]
    
    # mixing in a lucky one.. because sometimes anyone can get lucky ;)
    np.random.shuffle(population)
    parents.append(population[0])

    return parents


# Mixing too models by keeping their kernel weights intact
def kernelwise_mix(model_a, model_b):
    mix = []
    for i in range(len(model_a)):
        layer_a = model_a[i]
        layer_b = model_b[i]        
        # choosing kernels
        choice = np.random.randint(2, size = int(layer_a.size / layer_a.shape[-1])).reshape(layer_a.shape[:-1]).astype(bool)
        # extending the chosen kernel bools to the level of single values
        choice = np.repeat(choice, layer_a.shape[-1]).reshape(layer_a.shape)

        layer_mix = np.where(choice, layer_a, layer_b)
        mix.append(layer_mix)
        
    return mix

# Creates offsprings by mixing the layers of the model weights
def crossover(parent_models, offsprings_size):
    offsprings = []
    np.random.shuffle(parent_models)
    for k in range(offsprings_size):
        # Index of the first parent to mate.
        parent1_idx = k % len(parent_models)
        # Index of the second parent to mate.
        parent2_idx = (k+1) % len(parent_models)
        # mix of each modell kernelwise
        offsprings.append(kernelwise_mix(parent_models[parent1_idx], parent_models[parent2_idx]))
    return offsprings

def mutation(offsprings, mutation_chance=0.1, mutation_rate=1):

    # Mutation changes a single gene in each offspring randomly.
    for offspring in offsprings:
        for layer in offspring:
            trues = np.full(int(layer.size * mutation_chance), True)
            falses = np.full(layer.size - trues.size, False)
            mutation_indices = np.append(trues, falses)
            np.random.shuffle(mutation_indices)
            mutation_indices = mutation_indices.reshape(layer.shape)
                
            # The random value to be added to the gene.
            mutation_multiplier = np.random.normal(loc=0.0, scale=0.01 * mutation_rate, size=1)
            layer[mutation_indices] = layer[mutation_indices] + layer[mutation_indices] * mutation_multiplier
        
    return offsprings

In [ ]:
#population_fitness(model_topology_json, [weights], X_train, y_train)
#for i in range(100):
individual_fitness(None, weights, X_test, y_test)
#    if i % 5 == 0:
#        K.clear_session()

In [ ]:


population_size = 100 #sol_per_pop
num_parents_mating = 8
num_generations = 10000
mutation_chance = 0.01
mutation_rate = 5
reference_weights = weights
stuck_multiplier = 1
stuck_evasion_rate = 1.5

#Creating the initial population.
if os.path.isfile('genetic_learning_weights_only_kernel_faster_weights.checkpoint'):
    log.info('Resuming from previous checkpoint')
    with open('genetic_learning_weights_only_kernel_faster_weights.checkpoint', 'rb') as f:
        population_weights = pickle.load(f)
else:
    log.info('Creating random population')
    population_weights = []
    for _ in range(0, population_size):
        population_weights.append(create_model_programatically().get_weights())
    
    K.clear_session()

In [ ]:
fitness_scores = population_fitness(None, population_weights, X_test, y_test)
log.info("Accuracy of the best solution is : %f", max(fitness_scores))
log.info('Fitness scores of the last generation: |%s|', fitness_scores)

best_model = population_weights[np.argmin(fitness_scores)]

keras_model = create_model_programatically(best_model)
predictions = keras_model.predict_classes(X_test)
y_true = np.argmax(y_test, axis = 1)
accuracy = accuracy_score(y_true, predictions)
log.info('Best accuracy so far: %f', accuracy)

In [ ]:
if 'best_of_each_generation' in locals() or 'best_of_each_generation' in globals():
    pass
else:
    best_of_each_generation = []

for generation in tqdm(range(num_generations), desc='Generations progress', position=1):

    log.debug('Testing generation |%d| population: |%s|', generation, population_weights)
    # Measuring the fitness of each chromosome in the population.
    fitness_scores = population_fitness(None, population_weights, X_train, y_train)
    
    best_of_this_generation = max(fitness_scores)
    best_of_each_generation.append(best_of_this_generation)
    log.info("Best of geration |%d| has accuracy of |%f|", generation, best_of_this_generation)
    log.info('Fitness scores of this generation: |%s|', fitness_scores)
    
    
    # Selecting the best parents in the population for mating.
    parents = fittest_parents_of_generation(population_weights.copy(), fitness_scores, num_parents_mating)

    # Generating next generation using crossover.
    offsprings = crossover(parents.copy(), len(population_weights) - num_parents_mating)

    # Adding some variations to the offsrping using mutation.
    offsprings = mutation(offsprings, mutation_chance=mutation_chance * stuck_multiplier, mutation_rate=mutation_rate * stuck_multiplier)

    # Creating the new generation based on the parents and offspring.
    population_weights = []
    population_weights.extend(parents)
    population_weights.extend(offsprings)

    #cleanup resources
    K.clear_session()
    if generation > 0 and best_of_this_generation == best_of_each_generation[generation-1]:
        stuck_multiplier *= stuck_evasion_rate
        log.info('Stuck at local maximum, expanding mutation rate and chance by stuck multiplier of |%f|', stuck_multiplier)
    else:
        stuck_multiplier = 1

    if generation % 10 == 0:
        plot_learning(best_of_each_generation, num_generations)




In [ ]:
# Saving weights
with open('genetic_learning_weights_only_kernel_faster_weights.save','wb') as f:
    pickle.dump(population_weights, f)

In [ ]:
fitness_scores = population_fitness(None, population_weights, X_test, y_test)
log.info("Accuracy of the best solution is : %f", max(fitness_scores))
log.info('Fitness scores of the last generation: |%s|', fitness_scores)


In [ ]:
plot_learning(best_of_each_generation, num_generations)


In [ ]:
best_model = population_weights[np.argmin(fitness_scores)]

keras_model = create_model_programatically(best_model)
predictions = keras_model.predict_classes(X_test)

In [ ]:
unique, counts = np.unique(predictions, return_counts=True)
dict(zip(unique, counts))

In [ ]:
y_true = np.argmax(y_test, axis = 1)

In [ ]:
accuracy_score(y_true, predictions)

In [ ]:
import IPython
IPython.display.Audio("F:\\Tresorit\\01 - Startup Screen.mp3", autoplay=True)
